In [1]:
!git clone https://github.com/ElenaRyumina/EMO-AffectNetModel.git

Cloning into 'EMO-AffectNetModel'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 198 (delta 53), reused 46 (delta 46), pack-reused 138 (from 1)
Receiving objects: 100% (198/198), 200.85 MiB | 33.20 MiB/s, done.
Resolving deltas: 100% (104/104), done.


In [7]:
!pip install -r requirements.txt

ERROR: Invalid requirement: 'torch torchvision torchaudio': Expected end or semicolon (after name and no valid version specifier)
    torch torchvision torchaudio
          ^ (from line 8 of requirements.txt)


In [9]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


In [16]:
%cd /content/EMO-AffectNetModel

/content/EMO-AffectNetModel


In [21]:
!gdown --fuzzy https://drive.google.com/file/d/18e7ODzjngAd_UBf2_AwMRQ1Q6bts1s7E/view?usp=drive_link

Downloading...
From: https://drive.google.com/uc?id=18e7ODzjngAd_UBf2_AwMRQ1Q6bts1s7E
To: /content/EMO-AffectNetModel/RAVDESS_with_config.h5
100% 11.6M/11.6M [00:00<00:00, 33.4MB/s]


In [26]:
code = r"""
import argparse
import numpy as np
import pandas as pd
import os
import time
from scipy import stats
from functions import sequences
from functions import get_face_areas
from functions.get_models import load_weights_EE, load_weights_LSTM

import warnings
warnings.filterwarnings('ignore', category = FutureWarning)

parser = argparse.ArgumentParser(description="run")

parser.add_argument('--path_video', type=str, default='video/', help='Path to all videos')
parser.add_argument('--path_save', type=str, default='report/', help='Path to save the report')
parser.add_argument('--conf_d', type=float, default=0.7, help='Elimination threshold for false face areas')
parser.add_argument('--path_FE_model', type=str, default='models/EmoAffectnet/weights_0_66_37_wo_gl.h5',
                    help='Path to a model for feature extraction')
parser.add_argument('--path_LSTM_model', type=str, default='models/LSTM/RAVDESS_with_config.h5',
                    help='Path to a model for emotion prediction')

args = parser.parse_args()

def pred_one_video(path):
    start_time = time.time()
    label_model = ['Neutral', 'Happiness', 'Sadness', 'Surprise', 'Fear', 'Disgust', 'Anger']
    detect = get_face_areas.VideoCamera(path_video=path, conf=args.conf_d)
    dict_face_areas, total_frame = detect.get_frame()
    name_frames = list(dict_face_areas.keys())
    face_areas = list(dict_face_areas.values())
    EE_model = load_weights_EE(args.path_FE_model)
    LSTM_model = load_weights_LSTM(args.path_LSTM_model)
    features = EE_model(np.stack(face_areas))
    seq_paths, seq_features = sequences.sequences(name_frames, features)
    pred = LSTM_model(np.stack(seq_features)).numpy()
    all_pred = []
    all_path = []
    for id, c_p in enumerate(seq_paths):
        c_f = [str(i).zfill(6) for i in range(int(c_p[0]), int(c_p[-1])+1)]
        c_pr = [pred[id]]*len(c_f)
        all_pred.extend(c_pr)
        all_path.extend(c_f)
    m_f = [str(i).zfill(6) for i in range(int(all_path[-1])+1, total_frame+1)]
    m_p = [all_pred[-1]]*len(m_f)

    df=pd.DataFrame(data=all_pred+m_p, columns=label_model)
    df['frame'] = all_path+m_f
    df = df[['frame']+ label_model]
    df = sequences.df_group(df, label_model)

    if not os.path.exists(args.path_save):
        os.makedirs(args.path_save)

    filename = os.path.basename(path)[:-4] + '.csv'
    df.to_csv(os.path.join(args.path_save,filename), index=False)
    end_time = time.time() - start_time
    mode = stats.mode(np.argmax(pred, axis=1))[0]
    print('Report saved in: ', os.path.join(args.path_save,filename))
    print('Predicted emotion: ', label_model[mode])
    print('Lead time: {} s'.format(np.round(end_time, 2)))
    print()

def pred_all_video():
    path_all_videos = [
        f for f in os.listdir(args.path_video)
        if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))  # filter video files
    ]
    for id, cr_path in enumerate(path_all_videos):
        print('{}/{}'.format(id+1, len(path_all_videos)))
        pred_one_video(os.path.join(args.path_video, cr_path))


if __name__ == "__main__":
    pred_all_video()
"""
with open ("/content/EMO-AffectNetModel/run.py","w") as fin:
  fin.write(code)
print(f"Done")

Done


In [40]:
#edited for Cpu runtime
code = r"""
import argparse
import numpy as np
import pandas as pd
import os
import time
from scipy import stats
from functions import sequences
from functions import get_face_areas
from functions.get_models import load_weights_EE, load_weights_LSTM

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

parser = argparse.ArgumentParser(description="run")

parser.add_argument('--path_video', type=str, default='video/', help='Path to all videos')
parser.add_argument('--path_save', type=str, default='report/', help='Path to save the report')
parser.add_argument('--conf_d', type=float, default=0.7, help='Elimination threshold for false face areas')
parser.add_argument('--path_FE_model', type=str, default='models/EmoAffectnet/weights_0_66_37_wo_gl.h5',
                    help='Path to a model for feature extraction')
parser.add_argument('--path_LSTM_model', type=str, default='models/LSTM/RAVDESS_with_config.h5',
                    help='Path to a model for emotion prediction')

args = parser.parse_args()


def pred_one_video(path):
    start_time = time.time()
    label_model = ['Neutral', 'Happiness', 'Sadness', 'Surprise', 'Fear', 'Disgust', 'Anger']

    detect = get_face_areas.VideoCamera(path_video=path, conf=args.conf_d)
    dict_face_areas, total_frame = detect.get_frame()
    if total_frame == 0 or len(dict_face_areas) == 0:
        print(f"⚠️ Skipping {path} (no frames detected or unreadable video)")
        return

    name_frames = list(dict_face_areas.keys())
    face_areas = list(dict_face_areas.values())

    EE_model = load_weights_EE(args.path_FE_model)
    LSTM_model = load_weights_LSTM(args.path_LSTM_model)

    features = EE_model(np.stack(face_areas))
    seq_paths, seq_features = sequences.sequences(name_frames, features)

    pred = LSTM_model(np.stack(seq_features)).numpy()

    all_pred = []
    all_path = []
    for id, c_p in enumerate(seq_paths):
        c_f = [str(i).zfill(6) for i in range(int(c_p[0]), int(c_p[-1]) + 1)]
        c_pr = [pred[id]] * len(c_f)
        all_pred.extend(c_pr)
        all_path.extend(c_f)

    # fill missing frames with last prediction
    m_f = [str(i).zfill(6) for i in range(int(all_path[-1]) + 1, total_frame + 1)]
    m_p = [all_pred[-1]] * len(m_f)

    df = pd.DataFrame(data=all_pred + m_p, columns=label_model)
    df['frame'] = all_path + m_f
    df = df[['frame'] + label_model]
    df = sequences.df_group(df, label_model)

    if not os.path.exists(args.path_save):
        os.makedirs(args.path_save)

    filename = os.path.basename(path)[:-4] + '.csv'
    df.to_csv(os.path.join(args.path_save, filename), index=False)

    end_time = time.time() - start_time
    mode = stats.mode(np.argmax(pred, axis=1), keepdims=True)[0][0]
    print('✅ Report saved in:', os.path.join(args.path_save, filename))
    print('🎭 Predicted emotion:', label_model[mode])
    print('⏱️ Lead time: {} s'.format(np.round(end_time, 2)))
    print()


def pred_all_video():
    # only process valid video files
    valid_exts = ('.mp4', '.avi', '.mov', '.mkv')
    path_all_videos = [
        f for f in os.listdir(args.path_video)
        if f.lower().endswith(valid_exts)
    ]

    if not path_all_videos:
        print("⚠️ No valid video files found in", args.path_video)
        return

    for id, cr_path in enumerate(path_all_videos):
        print('{}/{}'.format(id + 1, len(path_all_videos)))
        pred_one_video(os.path.join(args.path_video, cr_path))


if __name__ == "__main__":
    pred_all_video()

"""
with open ("/content/EMO-AffectNetModel/run.py","w") as fin:
  fin.write(code)
print(f"Done")

Done


In [24]:
!pwd

/content/EMO-AffectNetModel


In [41]:
!python3 run.py --path_video "/content/Manya/" --path_save "/content/Manya/Out" --path_FE_model "/content/EMO-AffectNetModel/weights_0_66_49_wo_gl.h5" --path_LSTM_model "/content/EMO-AffectNetModel/RAVDESS_with_config.h5"

2025-09-12 14:49:25.461840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757688565.482025   19236 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757688565.488239   19236 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757688565.503664   19236 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757688565.503690   19236 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757688565.503693   19236 computation_placer.cc:177] computation placer alr

In [35]:
!nvtop

>